## Fill in Automatically

Fill in the missing columns of the data frame automatically. This is mostly information that is available online, or otherwise tedious to get.

In [1]:
import os

import pandas as pd
import numpy as np
import openpyxl

In [14]:
#Static
df_name = 'data_set_master_thesis_cala.xlsm'
lit_file_name = r'C:\Users\hso20\OneDrive\Plocha\IES\Diploma-Thesis\Literature\Literature.xlsx'
if not os.path.exists(df_name):
    raise ValueError('Missing data frame')

# Read the lit file
data_file = pd.read_excel(df_name, sheet_name = 'data_set')
lit_file = pd.read_excel(lit_file_name, sheet_name='Query literature')


In [44]:
def getCitations(active_sheet, lit_file:pd.DataFrame):
    '''Input the active excel_sheet, and the raw literature pandas data frame and return the desired array of citations.
    '''
    # Get the dictionary of citations for all studies
    rel_studies = lit_file[lit_file["Collected"] == "YES"]
    citations_dict = dict() # A list of dictionaries
    for study_iter in rel_studies.iterrows():
        study = study_iter[1]
        study_name = study["Label"]
        citations = study["Cited by"]
        citations_dict[study_name] = citations
    
    # Inside the active sheet, find the column indices for "study_name" and "citations"
    study_name_idx = None
    citations_idx = None

    for idx, cell in enumerate(active_sheet[1]):
        if cell.value == 'study_name':
            study_name_idx = idx
        elif cell.value == 'citations':
            citations_idx = idx

        if study_name_idx is not None and citations_idx is not None:
            break

    # Fill in the citations
    for row in active_sheet.iter_rows(min_row=2, values_only = True):
        study_name = row[study_name_idx]
        if study_name in citations_dict:
            cell = active_sheet.cell(row = row[0] + 1, column = citations_idx + 1) # Adding 1 because .cell() uses 1-indexing
            cell.value = citations_dict[study_name]
    
    return None

In [45]:
# Load the workbook and select the sheet
wb = openpyxl.load_workbook(df_name)
study_sheet = wb['data_set'] # Data_set worksheet

# Add the citations
getCitations(study_sheet, lit_file)

wb.save('updated_' + 'test_file.xlsx')